# Demo app

This demo app will launch a server at `localhot:8888` and expect an available Ollama server with the model `"qwen3:0.6b"`.
It is a Q&A chatbot interface to find opensource tools for your needs or project _(in french)_.

In [ ]:
import qA_Ap as qp

qp.init(
    ai="qwen3:0.6b",
    database="data/tools_db",
    api_server=8888
)

# Default app

The default setup launches an API server and frontend view on `localhot:8080`.

The database is a **FlatFileDB** on `"data/qaap_db"`.

The AI is an **OllamaAIInterface** with the model `"qwen3:1.7b"`.

In [ ]:
qp.init()

# Cloud app

The database is a **BaseRowFreeApiDB**.

The AI is an **CerebrasAIInterface** with the model `"llama3.1-8b"`.

In [ ]:
import json

import qA_Ap as qp
from qA_Ap.app.ai.interfaces.cerebras import CerebrasAIInterface
from qA_Ap.db.baserowfreeapidb import BaseRowFreeApiDB, BaseRowFreeApiDBTables

keys = {}

with open("keys.json", "r") as file:
    raw = file.read()
    keys = json.loads(raw)

ai = CerebrasAIInterface(
    key=keys["cerebras"], 
    model_name="llama3.1-8b"
)

tables = BaseRowFreeApiDBTables(
    documents="646003",
    documents_medias="650559",
    notes="648243",
    notes_medias="650572",
    summaries="648264",
    rag="648276"
)

db = BaseRowFreeApiDB(
    key=keys["baserow"],
    tables=tables
)

qp.init(
    database=db,
    ai=ai
)

# Manual database operations

Here, the app is not initialized. Only the database is setup.

The catalog is compiled manually then all documents with `couleurs` metadata containing the value or equal to `pink` are retrieved and displayed.

Then the attribute `"couleurs"` is indexed, its values are retrieved from the database and displayed.

In [2]:
from qA_Ap.db.flatfiledb import FlatFileDB
from qA_Ap.app.catalog import compile_catalog, compile_attribute, find_documents_by_metadata
from qA_Ap.globals import globals

globals.database = FlatFileDB("data/test_db")

compile_catalog()

results = find_documents_by_metadata(
    attribute="couleurs",
    value="pink"
)
for document in results:
    print(f"Found Document '{document["title"]}'",document["metadata"])

compile_attribute("couleurs")

couleurs = globals.database.get_attribute_values("couleurs")

print("All existing 'couleurs' values",couleurs.splitlines())

7 documents compiled successfully
Found Document 'Post pink' {'couleurs': 'pink'}
Found Document 'Post pink 2' {'couleurs': 'pink'}
Found Document 'Post pink and noir' {'couleurs': ['pink', 'noir']}
Found Document 'Post pink et rouge' {'couleurs': ['pink', 'rouge']}
All existing 'couleurs' values ['noir', 'bleu', 'pink', 'orange', 'rouge']


# Manual query

Here everything is setup manually for using the RAG functionalities manually. It is similar to using the init method with `api_server` set to `False`.

The vectostore is created and the query is done manually.

In [3]:
import json

import qA_Ap as qp
from qA_Ap.app.ai.methods import query, vectorize_documents
from qA_Ap.app.ai.faissvectorstore import FaissVectorStore
from qA_Ap.app.ai.interfaces.cerebras import CerebrasAIInterface
from qA_Ap.db.flatfiledb import FlatFileDB
from qA_Ap.globals import globals


keys = {}

with open("keys.json", "r") as file:
    raw = file.read()
    keys = json.loads(raw)

globals.database = FlatFileDB("data/tools_db")

globals.path_to_emmbeddings_model = "data/llms/Qwen3-Embedding-0.6B"

globals.ai_interface = CerebrasAIInterface(
        key=keys["cerebras"], 
        model_name="llama3.1-8b"
    )
globals.vectorstoreclass = FaissVectorStore

globals.system_prompt = qp.default_system_prompt

globals.object_of_search = "tools"

vectorize_documents()

print("=====================")
response = query("Je cherche un logiciel opensource pour pour créer un jeu avec mes propres modèles 3D.")
for chunk in response:
    print(chunk,end="",flush=True)
print("\n=====================")

27 documents
Document 'Amulet' divided into 1 chunks, Document 'Ardour' divided into 1 chunks, Document 'Aseprite' divided into 1 chunks, Document 'Audacity' divided into 1 chunks, Document 'Blender' divided into 1 chunks, Document 'Calibre' divided into 1 chunks, Document 'Darktable' divided into 1 chunks, Document 'Firefox' divided into 1 chunks, Document 'foobar2000' divided into 1 chunks, Document 'GIMP' divided into 1 chunks, Document 'Godot' divided into 1 chunks, Document 'Grav' divided into 1 chunks, Document 'Inkscape' divided into 1 chunks, Document 'Jitsi' divided into 1 chunks, Document 'kMeet' divided into 2 chunks, Document 'Krita' divided into 1 chunks, Document 'LibreOffice' divided into 1 chunks, Document 'LMMS' divided into 1 chunks, Document 'LÖVE' divided into 1 chunks, Document 'Nextcloud' divided into 1 chunks, Document 'OBS' divided into 1 chunks, Document 'Rufus' divided into 1 chunks, Document 'Scribus' divided into 1 chunks, Document 'Shotcut' divided into 1 c